#  Weekly project

## 1. Scraping the first page

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_url(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup
def scrape_tiki(url = 'https://tiki.vn/dien-thoai-may-tinh-bang/c1789?'):
    soup = get_url(url)
    items = soup.find_all('div',{"class": "product-item"})
    data = []
    for item in items:
        try: 
            dic = {"product_id":"","seller_id":"","title":"","price":"","image_url":""}
            dic["product_id"] = item["data-id"]
            dic["seller_id"] = item["data-seller-product-id"]
            dic["title"] = item["data-title"]
            dic["price"] = item["data-price"]
            if item.find("span",{"class":"image"}):
                dic["image_url"] = item.find("span",{"class":"image"}).img["src"]
            data.append(dic)
        except:
            print("We got an error")
    
    return (data)

In [ ]:
data = scrape_tiki()
data

In [ ]:
page1 = pd.DataFrame(data,columns = data[0].keys())

In [ ]:
page1.head()

## 2. Scraping all pages

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from random import randint

def get_url(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup
def scrape_tiki_all(url = "https://tiki.vn/dien-thoai-may-tinh-bang/c1789?"):
    url_base = url
    data = []
    page = 1
    items = True
    while items != []:
        soup = get_url(url)
        items = soup.find_all('div',{"class": "product-item"})
        for item in items:
            try: 
                dic = {"product_id":"","seller_id":"","title":"","price":"","image_url":""}
                dic["product_id"] = item["data-id"]
                dic["seller_id"] = item["data-seller-product-id"]
                dic["title"] = item["data-title"]
                dic["price"] = item["data-price"]
                if item.find("span",{"class":"image"}):
                    dic["image_url"] = item.find("span",{"class":"image"}).img["src"]
                data.append(dic)
            except:
                print("We got an error")
        print(page, len(data))
        page += 1
        url = url_base + "&page=" +str(page)
        time.sleep(randint(1,8))
    return data

In [ ]:
phone = scrape_tiki_all()

In [ ]:
phone_product = pd.DataFrame(phone,columns = phone[0].keys())

In [ ]:
phone_product.info()

In [ ]:
phone_product.head(10)

In [ ]:
phone_product.to_pickle("./phone_product.pkl")

In [ ]:
phone_product.to_csv("./phone_product.csv")

In [ ]:
unpickle_phone_product = pd.read_pickle("./phone_product.pkl")

In [ ]:
unpickle_phone_product.head()